In [33]:
import pandas as pd
import geopandas as gpd

user = 'aolsen'

# Comparison of MAZ data and TAZ-TM1 files when summing to county levels

## Load and process spatial reference data

In [34]:
maz_shape = gpd.read_file(
    f'/Users/{user}/Box/Modeling and Surveys/Urban Modeling/Spatial/Zones/v2.2/TM2_maz_taz_v2.2/mazs_TM2_v2_2.shp')
maz_shape = maz_shape.to_crs('EPSG:26910')

In [35]:
# get centroid
maz_shape['geom_pt'] = maz_shape.representative_point()

In [36]:
# get taz 1454 data
tazs = gpd.read_file(
    'https://services3.arcgis.com/i2dkYWmb4wHvYPda/arcgis/rest/services/transportation_analysis_zones_1454/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson').to_crs('EPSG:26910')
tazs.head(2)

,objectid,taz1454,district,county,gacres,Shape__Area,Shape__Length,geometry
0,1,566,11,Santa Clara,1708.683056,0.000444,0.097550,"POLYGON ((600337.262 4127333.574, 600264.066 4..."
1,2,565,11,Santa Clara,486.200669,0.000126,0.054171,"POLYGON ((601170.353 4126571.857, 601276.370 4..."


In [37]:
# join MAZ centroid to containing TAZ1454 shape
maz_x_taz_pt = gpd.sjoin(maz_shape.set_geometry(
    'geom_pt'), tazs[['taz1454', 'county', 'geometry']])

In [38]:
missing_taz_in_correspondence = set(range(1, 1455))-set(maz_x_taz_pt.taz1454)

## Load plan data

### MAZ data

In [39]:
pba_mazdata_2050 = pd.read_csv(
    f'/Users/{user}/Box/Modeling and Surveys/Development/Travel Model Two Development/Model Inputs/Land Use/FBP2050_by_TM2_maz_20220211.csv')
pba_mazdata_2050 = pba_mazdata_2050[pba_mazdata_2050.COUNTY.notna()]
pba_mazdata_2050.COUNTY = pba_mazdata_2050.COUNTY.astype(int)
pba_mazdata_2050['taz1454'] = pba_mazdata_2050.MAZ.map(
    maz_x_taz_pt.set_index('maz').taz1454)

In [40]:
pba_mazdata_2015 = pd.read_csv(
    f'/Users/{user}/Box/Modeling and Surveys/Development/Travel Model Two Development/Model Inputs/Land Use/FBP2015_by_TM2_maz_20220211.csv')
pba_mazdata_2015 = pba_mazdata_2015[pba_mazdata_2015.COUNTY.notna()]
pba_mazdata_2015.COUNTY = pba_mazdata_2015.COUNTY.astype(int)
pba_mazdata_2015['taz1454'] = pba_mazdata_2015.MAZ.map(
    maz_x_taz_pt.set_index('maz').taz1454)

### TAZ-1454 data

In [41]:
TAZ_PATH = f'/Users/{user}/Box/Modeling and Surveys/Share Data/plan-bay-area-2050/tazdata/PBA50_FinalBlueprintLandUse_TAZdata.xlsx'

pba_tazdata_2050 = pd.read_excel(TAZ_PATH, '2050')
pba_tazdata_2035 = pd.read_excel(TAZ_PATH, '2035')
pba_tazdata_2015 = pd.read_excel(TAZ_PATH, '2015')

### Comparison

In [42]:
def comparison(df1, df2, sumlev=['COUNTY']):

    return (df2.groupby(sumlev).sum()-df1.groupby(sumlev).sum().rename(columns=lambda x: x.upper())).stack()

In [43]:
pba_2050_diff_cnty = comparison(
    pba_mazdata_2050, pba_tazdata_2050, sumlev=['COUNTY'])
pba_2050_diff_cnty.unstack(1)

/var/folders/x8/3_n775lx7zq1nv3mddc7vmlh0000gp/T/ipykernel_10825/3387854450.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  return (df2.groupby(sumlev).sum()-df1.groupby(sumlev).sum().rename(columns=lambda x: x.upper())).stack()


,AGREMPN,FPSEMPN,HEREMPN,MWTEMPN,OTHEMPN,RETEMPN,TOTEMP,TOTHH
COUNTY,,,,,,,,
1,0.0,-3.0,-2.0,-3.0,-3.0,0.0,-11.0,-24.0
2,1.0,-4.0,-6.0,2.0,4.0,0.0,-3.0,25.0
3,-1.0,-115.0,-179.0,-5.0,-999.0,-1.0,-1300.0,-4.0
4,0.0,-1.0,-4.0,0.0,-3.0,-8.0,-16.0,-49.0
5,0.0,1.0,4.0,0.0,3.0,8.0,16.0,52.0
6,0.0,1.0,5.0,0.0,1.0,22.0,29.0,58.0
7,0.0,-1.0,-5.0,0.0,-1.0,-22.0,-29.0,-58.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
9,0.0,122.0,187.0,6.0,998.0,1.0,1314.0,-2.0
